# Optimizing with Cython

In some cases, it is possible to further improve performance using [Cython](http://www.cython.org), which is a Python-to-C/C++ compiler.  When looping over arrays of simple data types, it is possible to optimize out the array accesses (_e.g._, `x[i]`).

This page will not teach you how to use Cython, which is beyond the scope of what we can do here.  There are some examples in the fwdpy11 unit tests, though.

The examples here are generated in a Jupyter notebook using the Cython magics extension.  In reality, the Cython code would be offloaded into .pyx files.

In [1]:
%load_ext Cython

## The site-frequency spectrum

Let's revisit getting the sfs from a population.

In [2]:
%%cython --cplus --compile-args=-std=c++11
from libcpp.vector cimport vector
from libcpp.unordered_map cimport unordered_map
from libc.stdint cimport uint32_t,int8_t

#A first attempt at a Cython implementation.
#it takes a C++ vector as an argument,
#and an untype 2nd argument, which will
#be the mutations container
def sfs_cython(const vector[uint32_t] & counts,mutations):
    cdef size_t i = 0
    cdef nc = counts.size()
    cdef unordered_map[unsigned,unsigned] n,s
    while i < nc:
        if counts[i]>0:
            if mutations[i].neutral is True:
                n[counts[i]]+=1
            else:
                s[counts[i]]+=1
        i+=1
    return (n,s)

from cython.view cimport array as cvarray

#Let's be more aggressive, and write 
#a function taking memory views to
#mutation counts, their neutrality, 
#and effect sizes
def sfs_cython_memview(uint32_t[:] counts,int8_t[:] neutral,double[:] esizes):
    cdef size_t i = 0
    cdef nc = counts.shape[0]
    cdef unordered_map[unsigned,unsigned] n,s
    while i < nc:
        if counts[i]>0:
            if neutral[i]==1:
                n[counts[i]]+=1
            else:
                s[counts[i]]+=1
        i+=1
    return (n,s)

In [3]:
from collections import Counter
def sfs_np(pop):                                                  
    m = np.array(pop.mutations.array())      
    mc = np.array(pop.mcounts,copy=False)
    sfsn = Counter()             
    sfss = Counter()
    extant = np.where(mc>0)
    n=m['neutral']                                          
    for i in extant[0]:     
        if n[i]==1:                                                  
            sfsn[mc[i]] += 1                                             
        else:               
            sfss[mc[i]] += 1
    return (sfsn,sfss)

Run the same sims that we did before:

In [4]:
import fwdpy11
import fwdpy11.fitness
import fwdpy11.model_params
import fwdpy11.ezparams
import fwdpy11.wright_fisher
import numpy as np

rng = fwdpy11.GSLrng(42)

theta,rho = 100.0,100.0
pop = fwdpy11.SlocusPop(1000)

pdict = fwdpy11.ezparams.mslike(pop,simlen=pop.N,dfe=fwdpy11.regions.ExpS(0,1,1,-0.1,1),pneutral = 0.95)

params = fwdpy11.model_params.SlocusParams(**pdict)
fwdpy11.wright_fisher.evolve(rng,pop,params)

In [5]:
%timeit sfs_np(pop)

1.73 ms ± 13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit sfs_cython(pop.mcounts,pop.mutations)

5.97 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit sfs_cython_memview(pop.mcounts,np.array(pop.mutations.array())['neutral'],np.array(pop.mutations.array())['s'])

317 µs ± 5.56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
muts = np.array(pop.mutations.array())
%timeit sfs_cython_memview(pop.mcounts,muts['neutral'],muts['s'])

91.9 µs ± 1.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Nice--4x faster!

## Frequency vs. effect size

This is an example where Cython doesn't give a big payoff. We have to do a lot of work to get Cython to outperform what we can do "naturally" in fwdpy11 and the performance gains are marginal at best.

In [9]:
%%cython
from cython.view cimport array as cvarray
from libc.stdint cimport uint32_t
import numpy as np
cimport numpy as np
def freq_esize_cython(const uint32_t[:] counts, np.ndarray mutations):
    cdef size_t i=0,ei
    cdef nc = counts.shape[0]
    cdef np.ndarray[uint32_t,ndim=1] npcounts = np.array(counts,dtype=np.uint32,copy=False)
    cdef tuple extant = np.where((npcounts > 0) & (mutations['neutral']==0))
    cdef np.ndarray[double,ndim=2] x = np.zeros((extant[0].shape[0],2))    
    #The next three memory views
    #are needed to outperform a Python-only
    #implementation:
    cdef long[:] extview = extant[0]
    cdef double[:,:] view = x
    cdef double[:] sview = mutations['s']
    while i < extview.shape[0]:
        view[i][0] = counts[extview[i]]
        view[i][1] = sview[extview[i]]
        i+=1
    return x

In [10]:
%timeit freq_esize_cython(pop.mcounts,np.array(pop.mutations.array()))

146 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
def freq_esize_python(pop):
    npcounts = np.array(pop.mcounts)
    mutations = np.array(pop.mutations.array())
    extant = np.where((npcounts > 0) & (mutations['neutral']==0))
    x = np.zeros((extant[0].shape[0],2))    
    i=0
    for ei in extant[0]:
        x[i] = (npcounts[ei],mutations['s'][ei])
        i+=1
    return x

In [12]:
%timeit freq_esize_python(pop)

161 µs ± 5.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
